In [1]:
from xclib.data import data_utils
import numpy as np
import time
import pickle

In [2]:
#Change this later. Make the path as command line argument
X_train = data_utils.read_sparse_file('ass3_parta_data/train_x.txt', force_header=True)
train_class = np.genfromtxt('ass3_parta_data/train_y.txt').reshape(-1, 1)

X_valid = data_utils.read_sparse_file('ass3_parta_data/valid_x.txt', force_header=True)
valid_class = np.genfromtxt('ass3_parta_data/valid_y.txt').reshape(-1, 1)

X_test = data_utils.read_sparse_file('ass3_parta_data/test_x.txt', force_header=True)
test_class = np.genfromtxt('ass3_parta_data/test_y.txt').reshape(-1, 1)

/home/ayushi/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [3]:
f = open('ass3_parta_data/train_x.txt') 
m,n = f.readlines()[0].rstrip("\n").split(" ")
m,n = int(m),int(n)
f.close()

In [4]:
classes = np.unique(train_class)
pos_class_ind = np.where(train_class == 1)
neg_class_ind = np.where(train_class == 0)

In [5]:
attributes = np.array(range(X_train.A.shape[1]))

### Entropy H(Y) is calculated assuming Y only has two classes - 0 and 1

###### So we use log_2 and use pos and neg_num

In [6]:
def calc_entropyY(train_data, train_class):
    pos_num = train_data[np.where(train_class == 1)[0]].shape[0]
    neg_num = train_data[np.where(train_class == 0)[0]].shape[0]
    
    if (pos_num == 0) or (neg_num == 0):
        entr = 0 #in the limit
    else:
        entr = (pos_num/(pos_num+neg_num))*np.log2((pos_num+neg_num)/pos_num) + (neg_num/(pos_num+neg_num))*np.log2((pos_num+neg_num)/neg_num)
        
    return entr

In [7]:
#Entropy of Raw data
start = time.time()
print(calc_entropyY(X_train.A, train_class))
print("time = ", time.time()-start)

0.997775157550985
time =  0.08877849578857422


#### Reduction in Error is given by H(Y|X) by using value to split at the node
H(Y|X) 

       = H(Y|X=x)P(X=x)  -- find the two splits 

       = H(Y=y|X=x)P(X=x) -- find the entropy of both the splits. 

       = calc_entropy(split1, train_class_split1).P(X=x)
    

In [8]:
def calc_red_error(train_data, split, split_class, train_class):
    #print("In reduction of Error Module")
    #print(train_data.shape[0])
    error = 0
    for i in range(len(split)):
    #    print("Split shape = ", split[i].shape)
        error += (split[i].shape[0]/train_data.shape[0])*calc_entropyY(split[i], split_class[i])
    #print("Reduction in error = ", error)
    return error


In [9]:
def find_best_attr(train_data, train_class):
    #max_mutual_info = 0;   This doesnt work for the cases when mutual info becomes negative. 
    max_mutual_info = float("-inf");
    median = (np.median(train_data, axis=0))        
    #print(median)
    for i in range(len(attributes)):
        split = []
        split_class = []
        split.append(train_data[np.where(train_data[:,i] <= median[i])])
        split.append(train_data[np.where(train_data[:,i] > median[i])])
        split_class.append(train_class[np.where(train_data[:,i] <= median[i])])
        split_class.append(train_class[np.where(train_data[:,i] > median[i])])
        mutual_info = calc_entropyY(train_data, train_class) - calc_red_error(train_data, split, split_class, train_class)
        #print(mutual_info)
        if max_mutual_info < mutual_info:
            max_mutual_info = mutual_info
            best_attr = attributes[i]
            thres = median[i]
    return max_mutual_info, best_attr, thres

In [10]:
start = time.time()
print(find_best_attr(X_train.A, train_class))
print(time.time()-start)

(0.0793751756882577, 217, 0.0)
30.43116521835327


### Every Node  in the tree will have the following contents:

1. Left Subtree
2. Right Subtree
3. Attribute Name on which this node will split
4. Threshold of the attribute for left and right subtree (Median) - int value
5. Entropy
6. Whether this node is leaf or not - True/False
7. A final value of Y=0 or Y=1 in case of leaf

In [11]:
class newNode(object):
    def __init__(self, attr, Th, value):
        self.left = None
        self.right = None
        self.attribute = attr
        self.threshold = Th
        self.value = value
        self.leaf = False
        self.final = None
        self.root = False

In [12]:
def predict(tree, data):
    if tree.leaf:
        return tree.final
    if tree.left is None and tree.right is None:
        if tree.value[0] > tree.value[1]:
            return 0
        else:
            return 1
    if data[tree.attribute] <= tree.threshold:
        return predict(tree.left, data)
    elif data[tree.attribute] > tree.threshold:
        return predict(tree.right, data)


In [13]:
def calc_accuracy(tree, data, classes):
    print(tree.left)
    acc = 0
    for i in range(len(data)):
        pred = predict(tree, data[i])
        if (pred == classes[i]):
            acc+=1
    return acc/len(classes)

In [14]:
def growTree(train_data, classes, depth, depth_threshold=8):
    global num_nodes, nodes
    pos_num = train_data[np.where(classes == 1)[0]].shape[0]
    neg_num = train_data[np.where(classes == 0)[0]].shape[0]
    print("The number of nodes present in the graph = ", num_nodes)
    #print(neg_num, pos_num)
    
    if (depth == depth_threshold):
        num_nodes+=1
        leaf = newNode(None, None, None)
        leaf.leaf = True
        if (pos_num > neg_num):
            leaf.final = 1
        else:
            leaf.final = 0
        return leaf
            
    elif(pos_num == 0 or neg_num == 0):
        num_nodes+=1
        leaf = newNode(None, None, None)
        leaf.leaf = True
        if(pos_num != 0):
            leaf.final = 1
        elif(neg_num != 0):
            leaf.final = 0
        return leaf
        
    else:
        mutual_info, best_attr, thresh = find_best_attr(train_data, classes)
        print("Best Attribute with Threshold = ", best_attr, thresh)
        num_nodes+=1
        left_split = train_data[np.where(train_data[:,best_attr] <= thresh)]
        right_split = train_data[np.where(train_data[:,best_attr] > thresh)]
        left_split_class = classes[np.where(train_data[:,best_attr] <= thresh)]
        right_split_class = classes[np.where(train_data[:,best_attr] > thresh)]
        
        if (left_split.shape[0] == 0):
            leaf = newNode(None, None, None)
            leaf.leaf = True
            pos = left_split[np.where(left_split_class == 1)[0]].shape[0]
            neg = left_split[np.where(left_split_class == 0)[0]].shape[0]
            if (pos>neg):
                leaf.final = 1
            else:
                leaf.final = 0
            return leaf
        elif (right_split.shape[0] == 0):
            leaf = newNode(None, None, None)
            leaf.leaf = True
            pos = right_split[np.where(right_split_class == 1)[0]].shape[0]
            neg = right_split[np.where(right_split_class == 0)[0]].shape[0]
            if (pos>neg):
                leaf.final = 1
            else:
                leaf.final = 0
            return leaf
        else:
            addNode = newNode(best_attr, thresh, [neg_num, pos_num])
            print("The value of the class examples in this node = ", addNode.value)
            print("Building left tree on =", left_split.shape[0])
            addNode.left = growTree(left_split, left_split_class, depth+1)
            print("Building right tree in = ", right_split.shape[0])
            addNode.right = growTree(right_split, right_split_class, depth+1)
            return addNode

In [45]:
start = time.time()
num_nodes = 0
nodes = []
train_acc=[]; val_acc= []; test_acc = []
root = growTree(X_train.A, train_class, 0, 8)
print("Time Taken to grow tree = ", time.time() - start)

The number of nodes present in the graph =  0
Best Attribute with Threshold =  217 0.0
The value of the class examples in this node =  [30560, 34153]
Building left tree on = 47691
The number of nodes present in the graph =  1
Best Attribute with Threshold =  252 0.0
The value of the class examples in this node =  [17868, 29823]
Building left tree on = 43010
The number of nodes present in the graph =  2
Best Attribute with Threshold =  57 0.0
The value of the class examples in this node =  [17724, 25286]
Building left tree on = 39800
The number of nodes present in the graph =  3
Best Attribute with Threshold =  401 0.0
The value of the class examples in this node =  [17359, 22441]
Building left tree on = 37755
The number of nodes present in the graph =  4
Best Attribute with Threshold =  61 1.0
The value of the class examples in this node =  [17202, 20553]
Building left tree on = 19479
The number of nodes present in the graph =  5
Best Attribute with Threshold =  83 0.0
The value of the

Best Attribute with Threshold =  218 0.0
The value of the class examples in this node =  [259, 1219]
Building left tree on = 1417
The number of nodes present in the graph =  65
Best Attribute with Threshold =  27 0.0
The value of the class examples in this node =  [230, 1187]
Building left tree on = 725
The number of nodes present in the graph =  66
Best Attribute with Threshold =  108 0.0
The value of the class examples in this node =  [72, 653]
Building left tree on = 716
The number of nodes present in the graph =  67
Building right tree in =  9
The number of nodes present in the graph =  68
Building right tree in =  692
The number of nodes present in the graph =  69
Best Attribute with Threshold =  227 0.0
The value of the class examples in this node =  [158, 534]
Building left tree on = 432
The number of nodes present in the graph =  70
Building right tree in =  260
The number of nodes present in the graph =  71
Building right tree in =  61
The number of nodes present in the graph 

Best Attribute with Threshold =  128 0.0
The value of the class examples in this node =  [25, 68]
Building left tree on = 91
The number of nodes present in the graph =  144
Best Attribute with Threshold =  71 0.0
The value of the class examples in this node =  [23, 68]
Building left tree on = 64
The number of nodes present in the graph =  145
Best Attribute with Threshold =  87 2.0
The value of the class examples in this node =  [13, 51]
Building left tree on = 62
The number of nodes present in the graph =  146
Building right tree in =  2
The number of nodes present in the graph =  147
Building right tree in =  27
The number of nodes present in the graph =  148
Best Attribute with Threshold =  0 0.0
The value of the class examples in this node =  [10, 17]
Building left tree on = 25
The number of nodes present in the graph =  149
Building right tree in =  2
The number of nodes present in the graph =  150
Building right tree in =  2
The number of nodes present in the graph =  151
Buildin

Best Attribute with Threshold =  56 0.0
The value of the class examples in this node =  [845, 984]
Building left tree on = 1529
The number of nodes present in the graph =  216
Best Attribute with Threshold =  182 1.0
The value of the class examples in this node =  [823, 706]
Building left tree on = 838
The number of nodes present in the graph =  217
Best Attribute with Threshold =  43 0.0
The value of the class examples in this node =  [350, 488]
Building left tree on = 699
The number of nodes present in the graph =  218
Best Attribute with Threshold =  129 0.0
The value of the class examples in this node =  [249, 450]
Building left tree on = 525
The number of nodes present in the graph =  219
Best Attribute with Threshold =  109 0.0
The value of the class examples in this node =  [220, 305]
Building left tree on = 471
The number of nodes present in the graph =  220
Building right tree in =  54
The number of nodes present in the graph =  221
Building right tree in =  174
The number of 

Best Attribute with Threshold =  84 5.0
The value of the class examples in this node =  [2, 3]
Building left tree on = 3
The number of nodes present in the graph =  286
Building right tree in =  2
The number of nodes present in the graph =  287
Building right tree in =  38
The number of nodes present in the graph =  288
Best Attribute with Threshold =  359 22.0
The value of the class examples in this node =  [16, 22]
Building left tree on = 19
The number of nodes present in the graph =  289
Best Attribute with Threshold =  141 15.0
The value of the class examples in this node =  [3, 16]
Building left tree on = 11
The number of nodes present in the graph =  290
Building right tree in =  8
The number of nodes present in the graph =  291
Best Attribute with Threshold =  121 298.0
The value of the class examples in this node =  [3, 5]
Building left tree on = 4
The number of nodes present in the graph =  292
Best Attribute with Threshold =  47 0.0
The value of the class examples in this nod

In [30]:
print("The total number of nodes in the built tree = {}".format(num_nodes))

The total number of nodes in the built tree = 161


In [31]:
test_acc, nodes

([52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426,
  52.94144916786426],
 [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161])

In [24]:
train_acc = calc_accuracy(root, X_train.A, train_class)
val_acc = calc_accuracy(root, X_valid.A, valid_class)
test_acc = calc_accuracy(root, X_test.A, test_class)

Test Accuracy = 71.67%
Train Accuracy = 
Validation Accuracy = 

In [25]:
print("The Training Accuracy on Full-Grown Tree is = {:2.3f}%".format(train_acc*100))
print("The Vaidation Accuracy on Full-Grown Tree is = {:2.3f}%".format(val_acc*100))
print("The Test Accuracy on Full-Grown Tree is = {:2.3f}%".format(test_acc*100))

The Training Accuracy on Full-Grown Tree is = 74.004%
The Vaidation Accuracy on Full-Grown Tree is = 72.872%
The Test Accuracy on Full-Grown Tree is = 73.488%


##### Depth = 5
The Training Accuracy on Full-Grown Tree is = 68.46846846846847%

The Vaidation Accuracy on Full-Grown Tree is = 67.85184498423882%

The Test Accuracy on Full-Grown Tree is = 68.41592879328728%

##### Depth = 8
The Training Accuracy on Full-Grown Tree is = 74.004%

The Vaidation Accuracy on Full-Grown Tree is = 72.872%

The Test Accuracy on Full-Grown Tree is = 73.488%

##### Dept = 15
The Training Accuracy on Full-Grown Tree is = 79.90511952776103%

The Vaidation Accuracy on Full-Grown Tree is = 76.08937511589097%

The Test Accuracy on Full-Grown Tree is = 76.63529739001437%

##### Depth = 18
The Training Accuracy on Full-Grown Tree is = 81.199%

The Vaidation Accuracy on Full-Grown Tree is = 75.672%

The Test Accuracy on Full-Grown Tree is = 76.121%

##### Depth = 20
The Training Accuracy on Full-Grown Tree is = 81.423%

The Vaidation Accuracy on Full-Grown Tree is = 75.056%

The Test Accuracy on Full-Grown Tree is = 75.504%

In [5]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, train_class)

In [6]:
tree.plot_tree(clf) 

[Text(319.05051113061876, 366.52, 'X[217] <= 1.5\ngini = 0.498\nsamples = 64713\nvalue = [30560, 34153]'),
 Text(257.36407037011065, 360.35999999999996, 'X[363] <= 31.5\ngini = 0.468\nsamples = 47727\nvalue = [17872, 29855]'),
 Text(225.01246732417226, 354.2, 'X[57] <= 0.5\ngini = 0.489\nsamples = 41489\nvalue = [17697, 23792]'),
 Text(174.68336964363738, 348.03999999999996, 'X[401] <= 0.5\ngini = 0.496\nsamples = 38291\nvalue = [17335, 20956]'),
 Text(88.80787074189956, 341.88, 'X[151] <= 8.5\ngini = 0.499\nsamples = 36235\nvalue = [17178, 19057]'),
 Text(34.19124282323241, 335.71999999999997, 'X[107] <= 13.5\ngini = 0.474\nsamples = 14650\nvalue = [5639, 9011]'),
 Text(21.498475631824196, 329.55999999999995, 'X[106] <= 0.5\ngini = 0.493\nsamples = 10045\nvalue = [4437, 5608]'),
 Text(19.75143691552121, 323.4, 'X[21] <= 0.5\ngini = 0.491\nsamples = 9804\nvalue = [4240, 5564]'),
 Text(17.7520326703668, 317.23999999999995, 'X[353] <= 0.5\ngini = 0.489\nsamples = 9692\nvalue = [4130, 556